In [273]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from math import e

In [274]:
def forward(W1, W2, b1, b2, x):
    capa1 = W1 @ x + b1
    
    m = capa1.shape[0]
    sigmoide_c1 = np.zeros((5,1))
    
    for i in range(m):
        sigmoide_c1[i] = 1/(1 + e**(-capa1[i][0]))

    capa2 = W2 @ sigmoide_c1 + b2
    return capa2[0][0]


In [275]:
#Calculo del gradiente numerico
def funcion_objetivo(x, y, W1, W2, b1, b2):
    return 0.5*np.mean((forward(W1,W2,b1,b2,x.transpose()) - y)**2)
    # loss = 0.5*((forward(W1,W2,b1,b2,x.transpose())-y)**2)
    # return loss

def numerical_gradient(x, y, W1, W2, b1, b2, eps):
    for i in range(W1.shape[0]):
        for j in range(W1.shape[1]):
            W1_izq = W1 
            W1_izq[i][j] = W1_izq[i][j] + eps
            W1_der = W1 
            W1_der[i][j] = W1_der[i][j] - eps
            izq_W1 = funcion_objetivo(x, y, W1_izq, W2, b1, b2)
            der_W1 = funcion_objetivo(x, y, W1_der, W2, b1, b2)
            derivada_W1 = (izq_W1 - der_W1)/(2*eps)
    for i in range(W2.shape[0]):
        for j in range(W2.shape[1]):
            W2_izq = W2
            W2_izq[i][j] = W2_izq[i][j] + eps
            W2_der = W2
            W2_der[i][j] = W2_der[i][j] - eps
            izq_W2 = funcion_objetivo(x, y, W1, W2_izq, b1, b2)
            der_W2 = funcion_objetivo(x, y, W1, W2_der, b1, b2)
            derivada_W2 = (izq_W2 - der_W2)/(2*eps)
    for i in range(b1.shape[0]):
        for j in range(b1.shape[1]):
            b1_izq = b1
            b1_izq[i][j] = b1_izq[i][j] + eps
            b1_der = b1
            b1_der[i][j] = b1_der[i][j] - eps
            izq_b1 = funcion_objetivo(x, y, W1, W2, b1_izq, b2)
            der_b1 = funcion_objetivo(x, y, W1, W2, b1_der, b2)
            derivada_b1 = (izq_b1 - der_b1)/(2*eps)
    for i in range(b2.shape[0]):
        for j in range(b2.shape[1]):
            b2_izq = b2
            b2_izq[i][j] = b2_izq[i][j] + eps
            b2_der = b2
            b2_der[i][j] = b2_der[i][j] - eps
            izq_b2 = funcion_objetivo(x, y, W1, W2, b1, b2_der)
            der_b2 = funcion_objetivo(x, y, W1, W2, b1, b2_izq)
            derivada_b2 = (izq_b2 - der_b2)/(2*eps)
    
    gradiente = np.zeros((4,1))
    gradiente[0][0] = derivada_W1
    gradiente[1][0] = derivada_W2
    gradiente[2][0] = derivada_b1
    gradiente[3][0] = derivada_b2
    
    return gradiente


In [276]:
# #Calculo del gradiente numerico
# def funcion_objetivo(x, y, W1, W2, b1, b2):
#     loss = 0.5*((forward(W1,W2,b1,b2,x.transpose())-y)**2)
#     return loss

# def numerical_gradient(x, y, W1, W2, b1, b2, eps):

#     cincremental_der_W1 = funcion_objetivo(x, y, W1+eps, W2, b1, b2)
#     cincremental_izq_W1 = funcion_objetivo(x, y, W1-eps, W2, b1, b2)
#     derivada_W1 = (cincremental_der_W1-cincremental_izq_W1)/(2*eps)

#     cincremental_der_W2 = funcion_objetivo(x, y, W1, W2+eps, b1, b2)
#     cincremental_izq_W2 = funcion_objetivo(x, y, W1, W2-eps, b1, b2)
#     derivada_W2 = (cincremental_der_W2-cincremental_izq_W2)/(2*eps)

#     cicremental_der_b1 = funcion_objetivo(x, y, W1, W2, b1 + eps, b2)
#     cicremental_izq_b1 = funcion_objetivo(x, y, W1, W2, b1 - eps, b2)
#     derivada_b1 = (cicremental_der_b1 - cicremental_izq_b1)/(2*eps)

#     cicremental_der_b2 = funcion_objetivo(x, y, W1, W2, b1, b2 + eps)
#     cicremental_izq_b2 = funcion_objetivo(x, y, W1, W2, b1, b2 - eps)
#     derivada_b2 = (cicremental_der_b2 - cicremental_izq_b2)/(2*eps)

#     gradiente = np.zeros((4,1))
#     gradiente[0][0] = derivada_W1
#     gradiente[1][0] = derivada_W2
#     gradiente[2][0] = derivada_b1
#     gradiente[3][0] = derivada_b2
    
#     return gradiente

In [277]:
# funcion fit y loop de entrenamiento
def fit(x, y, W1, W2, b1, b2, learning_rate, epochs):
    eps = 1e-3
    loss_accum = []
    j = 0
    for i in range(epochs):
        aux_loss = 0
        for j in range(x.shape[0]):
            x_aux = np.array([x[j,:]])
            objective_value = funcion_objetivo(x_aux, y[j], W1, W2, b1, b2)
            aux_loss = aux_loss + objective_value

            gradiente = numerical_gradient(x_aux, y[j], W1, W2, b1, b2, eps)
            W1 = W1 - learning_rate*gradiente[0]
            W2 = W2 - learning_rate*gradiente[1]
            b1 = b1 - learning_rate*gradiente[2]
            b2 = b2 - learning_rate*gradiente[3]
        
        loss_accum.append(aux_loss/x.shape[0])

    theta = [W1, W2, b1, b2]
    return theta, loss_accum

In [278]:
#funcion fit y loop de entrenamiento

def fit2(x, y, W1, W2, b1, b2, learning_rate, epochs):
    eps = 1e-3
    loss_accum = []
    j = 0
    grad_w1 = np.zeros(W1.shape)
    grad_w2 = np.zeros(W2.shape)
    grad_b1 = np.zeros(b1.shape)
    grad_b2 = np.zeros(b2.shape)
    for i in range(epochs):
        aux_loss = 0
        for j in range(x.shape[0]):
            x_aux = np.array([x[j,:]])

            objective_value = funcion_objetivo(x_aux, y[j], W1, W2, b1, b2)

            aux_loss = aux_loss + objective_value

            gradiente = numerical_gradient(x_aux, y[j], W1, W2, b1, b2, eps)
            grad_w1 = grad_w1 + gradiente[0]
            grad_w2 = grad_w2 + gradiente[1]
            grad_b1 = grad_b1 + gradiente[2]
            grad_b2 = grad_b2 + gradiente[3]

        loss_accum.append(aux_loss/x.shape[0])

        grad_w1 = grad_w1 / x.shape[0]
        grad_w2 = grad_w2 / x.shape[0]
        grad_b1 = grad_b1 / x.shape[0]
        grad_b2 = grad_b2 / x.shape[0]

        W1 = W1 - learning_rate * grad_w1
        W2 = W2 - learning_rate * grad_w2
        b1 = b1 - learning_rate * grad_b1
        b2 = b2 - learning_rate * grad_b2

    theta = [W1, W2, b1, b2]
    return theta, loss_accum

In [279]:
def predict(x, W1, W2, b1, b2):
    y = forward(W1, W2, b1, b2, x)
    return y

### Inicialización de variables W1, b1, W2 y b2

In [280]:
W1 = np.random.random((5, 6))  # matriz 5x6 
b1 = np.random.random((5, 1))  # vector 5x1

W2 = np.random.random((1, 5))  # matriz 1x5
b2 = np.random.random((1, 1))  

### Recolección de datos

Tomamos los datos del archivo y los asignamos desde la posición 0 a la 315 como los de entrenamiento, y desde la 315 hasta el final como los datos de prueba.

In [281]:
df = pd.read_excel("Real estate valuation data set.xlsx")
datos = df.to_numpy()

X = datos[:100,[1,2,3,4,5,6]] 
Y = datos[:100,7]

X_test = datos[380:, [1,2,3,4,5,6]]
Y_test = datos[380:,7]

### Normalizamos las matrices

In [282]:
def normalizar_x(x):
    medias = x.mean()
    for i in range(x.shape[0]):
        x[i] = x[i] - medias
    return x  

def normalizar_y(y):
    media = y.mean()
    for i in range(y.shape[0]):
        y[i] = y[i] - media
    return y

x_norm = normalizar_x(X)
x_test_norm = normalizar_x(X_test)

### Entrenamiento de la red

In [283]:
theta, loss = fit(X, Y, W1, W2, b1, b2, 0.0005, 1000)

print(loss)

xt_test_norm = x_test_norm.transpose()

Xt_test = X_test.transpose()

predictions = []
resta = []

for i in range(X_test.shape[0]): 
    predictions.append(predict(xt_test_norm[:,i], theta[0], theta[1], theta[2], theta[3]))
    resta.append(predictions[i] - Y_test[i])
    print(predictions[i], "- ", Y_test[i], "=", resta[i])

# for i in range(X_test.shape[0]): 
#     predictions.append(predict(Xt_test[:,i], theta[0], theta[1], theta[2], theta[3]))
#     resta.append(predictions[i] - Y_test[i])
#     print(predictions[i], "- ", Y_test[i], "=", resta[i])


C:\Users\44591798\AppData\Local\Temp\ipykernel_35652\999334259.py:8: RuntimeWarning: overflow encountered in double_scalars
  sigmoide_c1[i] = 1/(1 + e**(-capa1[i][0]))


[713.5531256197221, 713.5531256197221, 713.5531256197221, 713.5531256197221, 713.5531256197221, 713.5531256197221, 713.5531256197221, 713.5531256197221, 713.5531256197221, 713.5531256197221, 713.5531256197221, 713.5531256197221, 713.5531256197221, 713.5531256197221, 713.5531256197221, 713.5531256197221, 713.5531256197221, 713.5531256197221, 713.5531256197221, 713.5531256197221, 713.5531256197221, 713.5531256197221, 713.5531256197221, 713.5531256197221, 713.5531256197221, 713.5531256197221, 713.5531256197221, 713.5531256197221, 713.5531256197221, 713.5531256197221, 713.5531256197221, 713.5531256197221, 713.5531256197221, 713.5531256197221, 713.5531256197221, 713.5531256197221, 713.5531256197221, 713.5531256197221, 713.5531256197221, 713.5531256197221, 713.5531256197221, 713.5531256197221, 713.5531256197221, 713.5531256197221, 713.5531256197221, 713.5531256197221, 713.5531256197221, 713.5531256197221, 713.5531256197221, 713.5531256197221, 713.5531256197221, 713.5531256197221, 713.5531256

In [ ]:
data = {'epochs': range(1000),
        'loss_acum': loss_acum}

df = pd.DataFrame(data)

df.plot(x='epochs', y='loss_acum')

plt.title('Funcion objetivo a lo largo del entrenamiento')
plt.xlabel('Epochs')
plt.ylabel('Loss Acumulated')

plt.show()